# Create a CPU Accelerated ResNet50 Inference Engine
In this section, we will create an inference engine wrapper, a class that will get image data as input, analyze it, and return the analysis result.

## Get Global Variables

First, read the previously stored variables. We need the name of the directory that will be used to store our ML solution files. If this directory does not exist, we will create a directory with a specified directory name.

In [ ]:
import sys
sys.path.append('../../../common')
from env_variables import *

import os
if not os.path.exists(lvaExtensionPath):
    os.mkdir(lvaExtensionPath)

## Download the ONNX ML Model
Next, we will download the sample ONNX model to use in the solution. Recall that in this sample, we will be using ResNet50, an image classification ML model. However, you may customize this ML model to fit your needs.

In [ ]:
# Do not change the values below, as the names are embedded into the score.py file.
# If the values are changed, you must also update the score.py content according to new file names
onnxModelFileName = "resnet50-v2-7.onnx"
onnxLabelFileName = "synset.txt"

onnxModelUrl = "https://github.com/onnx/models/raw/master/vision/classification/resnet/model/resnet50-v2-7.onnx"
onnxModelLabels = "https://github.com/onnx/models/raw/master/vision/classification/synset.txt"

The code snippet below downloads the ResNet50 model from the [ONNX Model Zoo](https://github.com/onnx/models) repository. 

> <span>[!NOTE]</span>
> The model download URLs frequently change, so if any of the code below fails, please update the source URLs accordingly.

In [ ]:
# Download the model files
import urllib.request
import os

# Download the ResNet50 pre-trained model
lvaExtensionFilePath = os.path.join(lvaExtensionPath, onnxModelFileName)
if not os.path.exists(lvaExtensionFilePath):
    res = urllib.request.urlretrieve(onnxModelUrl, lvaExtensionFilePath)
    print("Model file downloaded at: {}".format(lvaExtensionFilePath))
else:
    print("{} already exists here, so not downloading again.".format(lvaExtensionFilePath))
    
# Download the labels of the ResNet50 pre-trained model
lvaExtensionLabelFilePath = os.path.join(lvaExtensionPath, onnxLabelFileName)
if not os.path.exists(lvaExtensionLabelFilePath):
    res = urllib.request.urlretrieve(onnxModelLabels, lvaExtensionLabelFilePath)
    print("Labels file downloaded at: {}".format(lvaExtensionLabelFilePath))
else:
    print("{} already exists here, so not downloading again.".format(lvaExtensionLabelFilePath))

## Create Inference Engine Wrapper
Next, we will create a class that with different properties and methods to help scoring and analysing data from an image. This class will also help us specify analytics compute targets, such as CPU, VPU, FPGA, and debugging features.  

> <span style="color:red; font-weight: bold"> [!IMPORTANT] </span>
> Specific to this sample, we are using the ResNet50 model. As you can see from the code below,  
> * The ResNet50 model accepts only raw image bytes with 224 by 224 in size.  
> * Because we expect the `score` method to receive raw bytes of this size (224x224), we have statically coded the image size into our code. If the image is not 224x224, then the code will crash.
> * Why do we statically code the image size? LVA sends video frames to the `score` endpoint. In fact, LVA can send any image size and format. Since LVA can send images with 224x224 size, we do not need to spend additional compute cycles for re-sizing an image.

In [ ]:
%%writefile $lvaExtensionPath/score.py
import threading
import cv2
import numpy as np
import io
import onnxruntime
import json
import logging
import os
import linecache
import sys

logging.basicConfig(level=logging.DEBUG)

def PrintGetExceptionDetails():
    exType, exValue, exTraceback = sys.exc_info()

    tbFrame = exTraceback.tb_frame
    lineNo = exTraceback.tb_lineno
    fileName = tbFrame.f_code.co_filename

    linecache.checkcache(fileName)
    line = linecache.getline(fileName, lineNo, tbFrame.f_globals)

    exMessage = '[LVAX] Exception:\n\tFile name: {0}\n\tLine number: {1}\n\tLine: {2}\n\tValue: {3}'.format(fileName, lineNo, line.strip(), exValue)

    logging.info(exMessage)


class MLModel:
    def __init__(self):
        try:
            self._modelFileName = 'resnet50-v2-7.onnx'
            self._labelFileName = 'synset.txt'
            self._lock = threading.Lock()

            with open(self._labelFileName, "r") as f:
                self._labelList = [l.rstrip() for l in f]

            self._onnxSession = onnxruntime.InferenceSession(self._modelFileName)

        except:
            PrintGetExceptionDetails()

    def Preprocess(self, cvImage):
        try:
            imageBlob = cv2.cvtColor(cvImage, cv2.COLOR_BGR2RGB)
            imageBlob = np.array(imageBlob, dtype='float32')
            imageBlob /= 255.
            imageBlob = np.transpose(imageBlob, [2, 0, 1])
            imageBlob = np.expand_dims(imageBlob, 0)

            return imageBlob
        except:
            PrintGetExceptionDetails()

    def Postprocess(self, probabilities):
        try:
            sorted_prob = np.squeeze(np.sort(probabilities))[::-1]
            sorted_indices = np.squeeze(np.argsort(probabilities))[::-1]
            detectedObjects = []
                
            for i in range(3):
                confidence = sorted_prob[i]/100 #convert percent to decimal
                obj = self._labelList[sorted_indices[i]]
                obj_name = obj.split(' ', 1)[1]

                dobj = {
                    "type" : "classification",
                    "classification" : {
                        "tag" : {
                            "value" : obj_name, #skip the first word
                            "confidence" : confidence
                        }
                    }
                }
                detectedObjects.append(dobj)

            return detectedObjects
            
        except:
            PrintGetExceptionDetails()

    def Score(self, cvImage):
        try:
            with self._lock:
                imageBlob = self.Preprocess(cvImage)
                probabilities = self._onnxSession.run(None, {"data": imageBlob})
            
            return self.Postprocess(probabilities)

        except:
            PrintGetExceptionDetails()

The `score` method of the inference engine class above will return a dictionary of inferences in the following form:

```
{
    "type": "classification",
    "classification": {
        "tag": {
            "value": "llama",
            "confidence": 0.1784792137145996
        }
    }
},
{
    "type": "classification",
    "classification": {
        "tag": {
            "value": "gazelle",
            "confidence": 0.11214914321899414
        }
    }
},
{
    "type": "classification",
    "classification": {
        "tag": {
            "value": "wallaby, brush kangaroo",
            "confidence": 0.10473043441772462
        }
    }
}
```

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   